This package (censusify-philly) helps people to map census data (using the [census](https://pypi.org/project/census/) python package) to geographies in Philadelphia. We are using [shapely](https://pypi.org/project/shapely/) to manage the shapefile data.

**Before moving on, if you want to run this notebook, you will need to first acquire an API key for the US census data. You can do so [here](https://api.census.gov/data/key_signup.html).** Once you have this key, set it as an environment variable `CENSUS_API_KEY` or replace the variable below with the api key.

In [1]:
import os

census_api_key = os.environ["CENSUS_API_KEY"]

This specific notebook will explain how to map **block-group level demographics data** to Philadelphia **Police Service Areas**, using shapefiles provided by the city of Philadelphia. At the bottom of this notebook is some cleaned up code that then generates CSV files of demographics for PSAs, districts, and divisions.

This work will require the following pieces of information:
- from the us government census: 
  - shapefiles for all US census block groups in the city of Philadelphia
  - demographic counts for all US census block groups in the city of Philadelphia
- from the city of Philadelphia
  - shapefiles for all Police Service Areas in the city of Philadelphia
  
Lucky for us, both the city of Philadelphia and the US government use ArcGIS to store their shapefile data, so it will be returned in relatively similar formats.

## Step 1: Download the Census block group shapefiles

The shapefiles for the US census data are on a server called tigerweb. The following [URL](https://tigerweb.geo.census.gov/arcgis/rest/services/Census2020/Tracts_Blocks/MapServer/1/query) points to the Arcgis server where anyone can directly make calls that specifically return results at the census block group level.

In [2]:
base_url = "https://tigerweb.geo.census.gov/arcgis/rest/services/Census2020/Tracts_Blocks/MapServer/1/query"

There are several parameters to provide:

- **inSr/outSr**: The projection to return the data in. We want 4326, which is a reference to ESPG4326, which is the ESPG version of [WGS84](https://gisgeography.com/wgs84-world-geodetic-system/), which is the system that returns data in latitude and longitudes. 
- **where**: our query bounds. In our case we want to pass 'STATE=42 AND COUNTY=101' which represent PA and Philadelphia County respectively.
- **spatialRel**: How the results relate to the query. We want 'esriSpatialRelWithin' so we get all results that are within our query bounds.
- **f**: stands for format. We are returning it in JSON because thats the easiest to work with in python.
- **outFields**: The list of fields to return. We want all relevant location-related fields, as well as the lat/lng centroids.
- **returnGeometry**: Whether to return the geometry of the results. The geometry is a list of lat/lng pairs that when mapped represent the shape of a given geography.

In [3]:
import requests
from shapely.geometry import Polygon, Point
import matplotlib.pyplot as plt
from census import Census
import pandas as pd

# from censusify_philly.services.census.column_mappings import variable_race_mapping # or use appendix below
# from censusify_philly.services.census.column_mappings import variable_hispanic_mapping # or use appendix below

In [4]:
state = "42"  # PA
county = "101"  # Philly

params = {
    "inSr": 4326,
    "outSr": 4326,
    "spatialRel": "esriSpatialRelWithin",
    "outFields": "STATE,COUNTY,TRACT,BLKGRP,GEOID,CENTLAT,CENTLON",
    "f": "json",
    "where": f"STATE='{state}' AND COUNTY='{county}'",
    "geometryType": "esriGeometryPoint",
    "returnGeometry": True,
}

response = requests.get(base_url, params).json()

KeyboardInterrupt: 

In [ ]:
# An example of one of the census block groups.
response["features"][0]

It'll be easier if instead of a list of geographies, we store them as a dictionary, keyed by the GEOID (which is a concatenation of the state,county,tract, and block group). We store them below as `philly_census_block_group_geographies`

In [ ]:
philly_census_block_group_geographies = response["features"]

all_census_geographies_dict = {
    feat["attributes"]["GEOID"]: {
        "geometry": Polygon(feat["geometry"]["rings"][0]),
        "centroid": Point(
            float(feat["attributes"]["CENTLON"]),
            float(feat["attributes"]["CENTLAT"]),
        ),
    }
    for feat in philly_census_block_group_geographies
}

## Step 2: Download the Census demographics data

This step includes some data cleaning, as the raw census data gives non-descriptive column names. However, if you have census demographics data already cleaned, you can simply create a dictionary that is keyed on the GEOIDs (similar to the dictionary above, make sure to fill in zeroes similarly to how it is done from Arcgis).

I had to look up the census columns to get the column names that are used for the Census API. 

In [ ]:
race_cols = [f"P1_00{i}N" for i in range(1, 10)]
print("RACE:", race_cols)
hispanic_cols = ["P2_002N", "P2_003N"]
print("HISPANIC:", hispanic_cols)
race_hispanic_cols = [f"P2_0{i:02}N" for i in range(4, 12)]
print("RACE+HISPANIC:", race_hispanic_cols)
total_col = "H1_001N"
print("TOTAL:", total_col)

In [ ]:
census_demographics_fields = (
    [total_col] + race_cols + hispanic_cols + race_hispanic_cols
)
census_demographics_fields

In [ ]:
census_obj = Census(census_api_key)
census_demo_results = census_obj.pl.state_county_blockgroup(
    fields=["NAME"] + census_demographics_fields,
    state_fips=state,
    county_fips=county,
    tract="*",
    blockgroup="*",
)

I used this [link](https://api.census.gov/data/2020/dec/pl/variables.json) to create a more sensible mapping from these numerical columns (for example P2_006N) to descriptive column names. I also did some hacky manipulation of the column names to make them more readable.

In [ ]:
variables_response = requests.get(
    "https://api.census.gov/data/2020/dec/pl/variables.json"
)
census_variables = variables_response.json()["variables"]
relevant_census_variables = {key: val for key, val in census_variables.items() if key in census_demographics_fields}

In [ ]:
census_variables['P2_001N']

In [ ]:
for key,val in relevant_census_variables.items():
    print(key, val['label'])

In [ ]:
census_demo_results_df = pd.DataFrame(census_demo_results)

# combine to set full geoid as index
census_demo_results_df.index = census_demo_results_df[
    ["NAME", "state", "county", "tract", "block group"]
].apply(lambda x: f"{x['state']}{x['county']}{x['tract']}{x['block group']}", axis=1)

geoid_dict = {
    index: x[["NAME", "state", "county", "tract", "block group"]].to_dict()
    for index, x in census_demo_results_df.iterrows()
}

census_demo_results_df = census_demo_results_df.drop(["NAME","state","county","tract","block group"],axis=1)

In [ ]:
def census_demographics_mapping(census_p_number_dict, *, census_variables):
    cleaned_census_dict = {}
    for p_census_name, number_of_people in census_p_number_dict.items():
        human_readable_census_label = census_variables[p_census_name]["label"]
        cleaned_census_label = (
            human_readable_census_label.replace("!!Total:!!", "")
            .replace("Population of one race:!!", "")
            .replace("Population of two or more races", "Multiracial")
            .strip()
            .rstrip(":")
        )
        cleaned_census_dict[cleaned_census_label] = number_of_people

    return {
        # "White - Non-Latino": cleaned_census_dict['Not Hispanic or Latino:!!White alone'],
        # "Black - Non-Latino": cleaned_census_dict['Not Hispanic or Latino:!!Black or African American alone'],
        # "White - Latino": cleaned_census_dict['White alone'] - cleaned_census_dict['Not Hispanic or Latino:!!White alone'],
        # "Black - Latino": cleaned_census_dict['Black or African American alone'] - cleaned_census_dict['Not Hispanic or Latino:!!Black or African American alone'],
        "Hispanic or Latino": cleaned_census_dict["Hispanic or Latino"],
        "White": cleaned_census_dict["Not Hispanic or Latino:!!White alone"],
        "Black or African American": cleaned_census_dict[
            "Not Hispanic or Latino:!!Black or African American alone"
        ],
        "American Indian": cleaned_census_dict[
            "Not Hispanic or Latino:!!American Indian and Alaska Native alone"
        ],
        "Asian": cleaned_census_dict["Not Hispanic or Latino:!!Asian alone"]
        + cleaned_census_dict[
            "Not Hispanic or Latino:!!Native Hawaiian and Other Pacific Islander alone"
        ],
        "Unknown": cleaned_census_dict["Not Hispanic or Latino:!!Some Other Race alone"]
        + cleaned_census_dict["Not Hispanic or Latino:!!Multiracial"],
        "Total": cleaned_census_dict["Hispanic or Latino"]
        + cleaned_census_dict["Not Hispanic or Latino"],
    }    

cleaned_census_demo_results = []
for geoid, result in census_demo_results_df.iterrows():
    # Go through each demographic (i.e. P1_004N)
    odp_census_dict = census_demographics_mapping(result.to_dict(), census_variables=relevant_census_variables)
    odp_census_dict['geoid'] = geoid
    cleaned_census_demo_results.append(odp_census_dict)
cleaned_census_demo_results_df = pd.DataFrame(cleaned_census_demo_results).set_index(
    "geoid"
)

Next we take this list and we make a dictionary keyed on GEOID (similar to the census geography data)

In [ ]:
all_census_data_dict = {key["geoid"]: key for key in cleaned_census_demo_results}

## Step 3: Get PSA geography from the city

This query looks very similar to the Census geography query because it also uses Arcgis. Key differences are that we want all of the `outFields` so we pass a `*`. We also want all of the PSAs, so we pass where `1=1`.

In [ ]:
# PSA Geographies
psa_url = "https://services.arcgis.com/fLeGjb7u4uXqeF9q/arcgis/rest/services/Boundaries_PSA/FeatureServer/0/query"


state = "42"  # PA
county = "101"  # Philly

params = {
    "inSr": 4326,
    "outSr": 4326,
    "spatialRel": "esriSpatialRelWithin",
    "outFields": "*",
    "f": "json",
    "where": "1=1",
    "geometryType": "esriGeometryPoint",
    "returnGeometry": True,
}

all_psa_data = requests.get(psa_url, params).json()
all_psa_dict = {
    psa["attributes"]["PSA_NUM"]: psa["geometry"]["rings"][0]
    for psa in all_psa_data["features"]
}

## Step 4: Mapping census geographic data to PSAs

This example will go in detail on one PSA. The following will use District 14, PSA 4 as an example. This will be referenced as full PSA 144. 

We need to map the geography, and create weights for each census geography assigned to each PSA. These weightings are then used to assign demographics data.

There are two methods that can be used to do this mapping. 

1) By block group centroid
2) By % area overlap

In [ ]:
psa_name = "144"
psa_x = Polygon(all_psa_dict[psa_name])
psa_x

### Step 4, Option 1: Map census geography data to PSAs (By Block Group centroid)

This is a method commonly used in research papers that assigns the full demographics from all census block groups that have centroids inside of a PSA.

In [ ]:
census_block_groups_by_centroid = {
    census_name: 1
    for census_name, census_x in all_census_geographies_dict.items()
    if census_x["centroid"].within(psa_x)
}
census_block_groups_by_centroid

The plot below is a grey area (representing the PSA) overlayed with a red area (the census block groups that are associated with that PSA). The lines represent the boundaries of the census block groups.

It can be seen that the plot below has some gray missing area (where part of the PSA isn't being overlapped by any census block group), and some light pink area of one of the census block groups are partially outside the PSA. These mismatches may lead to errors in the demographic mapping step.

In [ ]:
for census_block_group, weight in census_block_groups_by_centroid.items():
    plt.fill(
        *all_census_geographies_dict[census_block_group]["geometry"].exterior.xy,
        label=census_block_group,
        color="red",
        alpha=0.5
    )
plt.fill(*psa_x.exterior.xy, label=psa_name, color="black", alpha=0.2)

### Step 4, Option 2: Map census geography data to PSAs (By % area covered)

In this method, we find all of the census block groups that overlap in some way to the given PSA and calculate what % of that Census Block Group overlaps with the PSA. There are some rounding errors with census block groups that touch but don't overlap with PSAs, so we remove Census Block groups with a calculated area that is very small (because that indicates they simply are touching).

The following code also calculates the percent area of each given Census Block Group that overlaps with the PSA. This will later be used to weight the demographics data from each Census Block Group to a given PSA. 

In [ ]:
census_block_groups_pct_area = {
    census_name: census_x["geometry"].intersection(psa_x).area
    / census_x["geometry"].area
    for census_name, census_x in all_census_geographies_dict.items()
    if census_x["geometry"].intersects(psa_x)
    and census_x["geometry"].intersection(psa_x).area >= 0.000001
}
census_block_groups_pct_area

 The plot below is a grey area (representing the PSA) overlayed with a red area (the census block groups that are associated with that PSA). The lines represent the boundaries of the census block groups.
 
 While it looks like an additional census block group (421019801001, the light pink one at the bottom of the plot) was added that only slightly overlaps with the given PSA, this is accounted for because its weight is 0.16 (as can be seen in the last line of the dictionary above), meaning that this method assumes that only 16% of the population of that census block group live in the PSA.

In [ ]:
for census_block_group, weight in census_block_groups_pct_area.items():
    plt.fill(
        *all_census_geographies_dict[census_block_group]["geometry"].exterior.xy,
        label=census_block_group,
        color="red",
        alpha=0.5
    )
plt.fill(*psa_x.exterior.xy, label=psa_name, color="black", alpha=0.2)

## Step 5: Assign census demographic data to PSAs

Given a dictionary (where the keys are the block groups that we think are in that PSA, and the values are the weights), we can then assign demographics data to PSAs.

We will calculate the demographics for the prior PSA using both methods.


In [ ]:
census_pct = census_block_groups_pct_area

The following shows the demographics for each Census Block Group that we previously said is associated with a given PSA.

In [ ]:
census_series = pd.Series(census_pct)
census_series = census_series[census_series > 0]

demographics_of_overlapping_block_groups_df = (
    pd.DataFrame(all_census_data_dict).T.drop("geoid", axis=1).loc[census_series.index]
)
demographics_of_overlapping_block_groups_df

Next (and this is only relevant for the method using the % area method), we multiply the number of people in a given census block group by the % area of the Census Block Group that overlaps with the PSA to get the estimated number of people in each Census Block Group that are also in that PSA. 

As you can see below, the numbers are similar to the above for most Census Block Groups, with the exception of the last row, which we previously saw only had 16% overlap. Above shows 37 White Non-Latino people live in Census Block Group 421019801001, but we belive that only 6 of those people live in the PSA.

In [ ]:
demographics_of_overlapping_block_groups_df.mul(census_series, axis=0)

After summing all of the people across all the associated census block groups in the given PSA, we round to make sure we get even numbers of people. We then end up with the demographics of the PSA.

In [ ]:
demographics_of_overlapping_block_groups_df.mul(census_series, axis=0).sum().astype(
    float
).round().to_dict()

Lets see how this compares to the centroid method. It seems like the % area method includes 2 more 'Black Non-Latino' people and 4 more 'White - Non-Latino' people.

In [ ]:
census_pct_by_centroid = census_block_groups_by_centroid
census_series_by_centroid = pd.Series(census_pct_by_centroid)
census_series_by_centroid = census_series_by_centroid[census_series_by_centroid > 0]

pd.DataFrame(all_census_data_dict).T.drop("geoid", axis=1).loc[
    census_series_by_centroid.index
].mul(census_series, axis=0).sum().astype(float).round().to_dict()

## The full code for all PSAs, Districts, Divisions, and the City using some cleaned up functions.

In [ ]:
pip install censusify-philly

In [26]:
from censusify_philly.census.models import CensusDataQuery
from censusify_philly.arcgis.models import ArcgisQuery
from censusify_philly.police_geographies import PoliceDataCensusDemographicsResult, CENSUS_BLOCK_GROUP_ARCGIS_QUERY_SOURCE, OPEN_DATA_PHILLY_ARCGIS_QUERY_SOURCES

census_data_query = CensusDataQuery(census=Census(census_api_key))
census_demographics_results = census_data_query._get_demographic_data(
    state_fips="42", # PA 
    county_fips="101" # Philly
)
census_demo_data_df = PoliceDataCensusDemographicsResult.as_df(
    census_demographics_results
)

census_arcgis_query = ArcgisQuery(CENSUS_BLOCK_GROUP_ARCGIS_QUERY_SOURCE)
for geography in OpenDataPhillyGeographyName:
    other_arcgis_query = ArcgisQuery(
        OPEN_DATA_PHILLY_ARCGIS_QUERY_SOURCES[geography.value]
    )
    matcher = CensusGeoMatcher(
        census_arcgis_query=census_arcgis_query,
        other_arcgis_query=other_arcgis_query,
    )

    # Download the geographic data
    print(f"Downloading geographic data for {geography}...")
    geo_results = matcher.generate_geo_matched_results(
        state_fips=STATE_FIPS,
        county_fips=COUNTY_FIPS,
        relationship=CensusBlockRelationship.centroid_is_within,
    )
    df = matcher.assign_demographic_data_to_custom_geographies(
        geo_results=geo_results, census_demographics_df=census_demo_data_df
    )
    df.sort_index().to_csv(f"{geography.value}.csv")


NameError: name 'OpenDataPhillyGeographyName' is not defined